### The Data

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/stars.csv')

In [4]:
df.sample(5)

,description,language,repo,stargazers,user
24405,Python MUD/MUX/MUSH/MU* development system,Python,evennia/evennia,988,ka1nsha
23961,Ready-to-run Docker images containing Jupyter ...,Shell,jupyter/docker-stacks,4365,sseemayer
3137,The next generation relational database.,Python,edgedb/edgedb,3293,Na0ture
22593,Distributed crawler powered by Headless Chrome,JavaScript,yujiosaka/headless-chrome-crawler,4276,andrewwxy
11412,A repository for learning various heap exploit...,C,shellphish/how2heap,3330,owen800q


### The Cleanup

In [5]:
df = df[df['repo'] != 'maxhumber/gazpacho']
df = df[df.language.isin(['Python', 'Jupyter Notebook'])]
popular = pd.DataFrame(df['repo'].value_counts())
select_repos = popular[popular['repo'] >= 5].index.tolist()
df = df[df['repo'].isin(select_repos)]

In [6]:
df.sample(5)

,description,language,repo,stargazers,user
13273,"Transistor, a Python web scraping framework fo...",Python,bomquote/transistor,188,erdimollahuseyin
5007,👨‍💼Python Wrapper for the Linkedin API,Python,tomquirk/linkedin-api,155,janscas
6881,"More routines for operating on iterables, beyo...",Python,erikrose/more-itertools,1061,DahlitzFlorian
13146,Kubernetes-native Deep Learning Framework,Python,sql-machine-learning/elasticdl,315,erdimollahuseyin
15125,Open Source Differentiable Computer Vision Lib...,Python,arraiyopensource/kornia,1233,mattias71


In [7]:
df.shape

(2229, 5)

In [8]:
# shove into something that looks like this?
pd.DataFrame([
    [0, 0, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 0, 0, 1]
])

,0,1,2,3
0,0,0,1,0
1,0,1,1,0
2,1,0,0,1
3,0,0,0,1


### The Preparation

In [9]:
df = df.groupby(['user'])['repo'].apply(lambda x: ','.join(x))
df = pd.DataFrame(df)

In [10]:
df.sample(5)

,repo
user,
fly51fly,"sebst/pythonic-news,pytorch/captum,wxWidgets/P..."
boada,"kootenpv/brightml,hukkelas/DeepPrivacy,Phantom..."
tones7778,"prabhupant/python-ds,plainas/flask-swagger-typ..."
rlgomes,"svip-lab/impersonator,timothycrosley/hypothesi..."
edujtm,"flask-restful/flask-restful,encode/starlette,j..."


In [11]:
df.loc['Curlybear'].values.tolist()

['vinta/awesome-python,CorentinJ/Real-Time-Voice-Cloning,danijar/handout,ageitgey/face_recognition,jofpin/trape,grapheneX/grapheneX,gto76/python-cheatsheet,trekhleb/homemade-machine-learning,manrajgrover/halo,chris104957/maildown,psf/black,stewartmcgown/uds,TheAlgorithms/Python,cool-RR/PySnooper,Bogdanp/molten,nicolargo/glances,mkaz/termgraph,donnemartin/system-design-primer,chubin/cheat.sh,psf/requests-html,Zulko/moviepy,jarun/Buku,pypa/pipenv,soimort/you-get,public-apis/public-apis,HelloZeroNet/ZeroNet']

### The Model

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


class NNRecommender:
    def __init__(self, n_neighbors=10, max_features=1000, tokenizer=lambda x: x.split(",")):
        self.cv = CountVectorizer(tokenizer=tokenizer, max_features=max_features)
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)

    def fit(self, X):
        self.X = X
        X = self.cv.fit_transform(X)
        self.nn.fit(X)
        return self

    def predict(self, X):
        Xp = []
        for Xi in X:
            Xt = self.cv.transform([Xi])
            neighbors = self.nn.kneighbors(Xt, return_distance=False)
            repos = []
            for n in neighbors[0]:
                r = self.X.iloc[int(n)].split(",")
                repos.extend(r)
            repos = list(set(repos))
            repos = [r for r in repos if r not in Xi.split(",")]
            Xp.append(repos)
        return Xp

In [13]:
model = NNRecommender()
model.fit(df['repo'])

In [14]:
df['repo'][42]

'CorentinJ/Real-Time-Voice-Cloning,s0md3v/goop,psf/requests,slundberg/shap,facebookresearch/pytext,practicalAI/practicalAI,Zulko/moviepy,python/mypy,ParthS007/background,keon/algorithms,google/python-fire,eriklindernoren/ML-From-Scratch,minimaxir/big-list-of-naughty-strings,pudo/dataset,nvbn/thefuck,dbcli/pgcli,faif/python-patterns,chriskiehl/Gooey'

In [15]:
model.predict([df['repo'][42]])

[['Miserlou/Zappa',
  'ageron/handson-ml2',
  'donnemartin/interactive-coding-challenges',
  'sherlock-project/sherlock',
  'scikit-learn/scikit-learn',
  'smacke/subsync',
  'dae/anki',
  'weskerfoot/DeleteFB',
  'sloria/TextBlob',
  'shengqiangzhang/examples-of-web-crawlers',
  'deepfakes/faceswap',
  'fastai/fastai']]

### Under the Hood

In [16]:
df.iloc[0].values.tolist()

['python/cpython,hugapi/hug,HypothesisWorks/hypothesis,timothycrosley/portray,timothycrosley/hypothesis-auto,tiangolo/fastapi,bocadilloproject/bocadillo,tartiflette/tartiflette,encode/httpx,pytest-dev/pytest,tonybaloney/wily,sdispater/poetry,mahmoud/glom,psf/black,chubin/wttr.in']

In [17]:
cv = CountVectorizer(tokenizer=lambda x: x.split(','), max_features=1000)
X = cv.fit_transform(df['repo'])
X.todense()[:5]

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [18]:
nn = NearestNeighbors(n_neighbors=10)

In [19]:
nn.fit(X)
ind = nn.kneighbors(return_distance=False)
ind[:5]

array([[106,  14,  99,  48,  65,   3,  10,  19,   2,  40],
       [ 14,  66,  99,  10,  19,  65,  48,   3,  40,  12],
       [ 10,  19,   3,  99,  48,  14,  65,  40,  66,  92],
       [ 48,  10,  99,  19,  14,  65,   2,  40,  92,  72],
       [ 28,  10,  48,  72,  20,  19,  99,  14,  65,   3]])

In [20]:
df.iloc[106].values

array(['timothycrosley/hypothesis-auto,timothycrosley/portray,sdispater/poetry,wyounas/homer,tiangolo/fastapi,psf/black,marshmallow-code/marshmallow,falconry/falcon,cookiecutter/cookiecutter'],
      dtype=object)

#### Pickle 

In [21]:
import pickle

In [22]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

PicklingError: Can't pickle <function NNRecommender.<lambda> at 0x1a1d745200>: attribute lookup NNRecommender.<lambda> on __main__ failed

In [23]:
import dill # pip install dill

with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

del model

with open('model.pkl', 'rb') as f:
    model = dill.load(f)

In [24]:
model.predict(['streamlit/streamlit,huggingface/transformers,encode/httpx,aws/chalice,maxhumber/chart'])

[['minimaxir/big-list-of-naughty-strings',
  'vinta/awesome-python',
  'plasticityai/supersqlite',
  'sloria/TextBlob',
  'Avik-Jain/100-Days-Of-ML-Code',
  'donnemartin/system-design-primer',
  'shengqiangzhang/examples-of-web-crawlers',
  'mherrmann/fbs',
  'scikit-learn/scikit-learn',
  'TheAlgorithms/Python',
  'ytdl-org/youtube-dl',
  'PySimpleGUI/PySimpleGUI',
  'fastai/fastai']]